In [4]:
import matplotlib.gridspec as gridspec
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize, LogNorm
import scqubits
from tqdm import tqdm
from IPython.display import clear_output
import qutip
from functools import partial
import matplotlib
from CoupledQuantumSystems.drive import *
from scipy.optimize import minimize
import multiprocessing
import pickle

from CoupledQuantumSystems.noise import (
    first_order_derivative,
    second_order_derivative,
    get_frequency,
    diel_spectral_density,
    one_over_f_spectral_density,
    T_phi,
)
from CoupledQuantumSystems.IFQ import gfIFQ
from CoupledQuantumSystems.evo import ODEsolve_and_post_process

# -------------------------------
# Define your parameters and objects
# -------------------------------
EJ = 3
EJoverEC = 6
EJoverEL = 25
EC = EJ / EJoverEC
EL = EJ / EJoverEL

qbt = gfIFQ(EJ=EJ, EC=EC, EL=EL, flux=0, truncated_dim=20)
e_ops = [
    qutip.basis(qbt.truncated_dim, i) * qutip.basis(qbt.truncated_dim, i).dag()
    for i in range(10)
]

element = np.abs(
    qbt.fluxonium.matrixelement_table("n_operator", evals_count=3)[1, 2]
)

# -------------------------------
# Objective function
# -------------------------------
def objective(x, t_tot):
    """
    x    = [amp, w_d]
    t_tot: fixed total time
    """
    amp, w_d = x
    tlist = np.linspace(0, t_tot * 1.15, 100)

    initial_states = [
        qutip.basis(qbt.truncated_dim, 1),
        qutip.basis(qbt.truncated_dim, 2),
    ]

    drive_terms = [
        DriveTerm(
            driven_op=qutip.Qobj(qbt.fluxonium.n_operator(energy_esys=True)),
            pulse_shape_func=square_pulse_with_rise_fall,
            pulse_id="pi",
            pulse_shape_args={
                "w_d": w_d,    # No extra 2pi factor
                "amp": amp,    # No extra 2pi factor
                "t_square": t_tot * 0.85,
                "t_rise": t_tot * 0.15,
            },
        )
    ]

    # Solve the dynamics for each initial state
    results = []
    for init_state in initial_states:
        res = ODEsolve_and_post_process(
            y0=init_state,
            tlist=tlist,
            static_hamiltonian=qbt.diag_hamiltonian,
            drive_terms=drive_terms,
            e_ops=e_ops,
        )
        results.append(res)

    # Calculate final populations
    # (We want to transfer |1> -> |2> and |2> -> |1>.)
    one_minus_pop2 = abs(1 - results[0].expect[2][-1])  # from state |1> to |2>
    one_minus_pop1 = abs(1 - results[1].expect[1][-1])  # from state |2> to |1>

    return one_minus_pop2 + one_minus_pop1


# -------------------------------
# Function to run optimization for a given t_tot
# -------------------------------
def run_optimization_for_ttot(t_tot):
    """
    For a fixed t_tot, optimize over [amp, w_d].
    Returns a dict with:
      {
        't_tot': t_tot,
        'result': result_of_minimize (object),
        'optimal_amp': result_of_minimize.x[0],
        'optimal_w_d': result_of_minimize.x[1],
        'optimal_fun': result_of_minimize.fun
      }
    """
    # A naive initial guess for [amp, w_d]:
    #   amp from your earlier logic: np.pi / element / t_tot / 3.1415 / 2
    #   w_d from your earlier logic: qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]
    init_amp = np.pi / element / t_tot / 3.1415 / 2
    init_wd = qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]
    initial_guess = [init_amp, init_wd]

    # Minimize
    res = minimize(
        lambda x: objective(x, t_tot),
        x0=initial_guess,
        method="Nelder-Mead",
    )

    return {
        "t_tot": t_tot,
        "result": res,
        "optimal_amp": res.x[0],
        "optimal_w_d": res.x[1],
        "optimal_fun": res.fun,
    }


# -------------------------------
# Main parallel loop
# -------------------------------
# Example list of t_tot values
t_tot_list = np.linspace(50,250,21)

# Use multiprocessing Pool to parallelize
with multiprocessing.Pool() as pool:
    results = pool.map(run_optimization_for_ttot, t_tot_list)

# Save to pickle
with open("optimized_results.pkl", "wb") as f:
    pickle.dump(results, f)

print("Optimization complete. Results saved to 'optimized_results.pkl'.")


 *********97%********** ] Elapsed 6.98s / Remaining 00:00:00:000[          1%           ] Elapsed 0.12s / Remaining 00:00:00:11[          2%           ] Elapsed 0.16s / Remaining 00:00:00:07[*         3%           ] Elapsed 0.18s / Remaining 00:00:00:05[*         4%           ] Elapsed 0.20s / Remaining 00:00:00:04[*         5%           ] Elapsed 0.26s / Remaining 00:00:00:04[*         6%           ] Elapsed 0.27s / Remaining 00:00:00:04[**        7%           ] Elapsed 0.28s / Remaining 00:00:00:03[**        8%           ] Elapsed 0.30s / Remaining 00:00:00:03[**        9%           ] Elapsed 0.37s / Remaining 00:00:00:03[**       10%           ] Elapsed 0.43s / Remaining 00:00:00:03[***      11%           ] Elapsed 0.47s / Remaining 00:00:00:03[***      12%           ] Elapsed 0.52s / Remaining 00:00:00:03[***      13%           ] Elapsed 0.56s / Remaining 00:00:00:03[***      14%           ] Elapsed 0.70s / Remaining 00:00:00:04[***      15%           ] Elapsed 0.82s / Remaining 00